In [1]:
# This cell connects to google drive, authenticates connection, and iterates over file list displaying each file's title and ID.
# it also imports and installs all the necessary libraries
!pip install -U -q PyDrive
!pip install -U -q wordcloud
!pip install -U -q bokeh
!pip install textblob
!pip install sklearn

import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

from bs4 import BeautifulSoup

from wordcloud import WordCloud
from scipy.stats import hmean
from scipy.stats import norm
from pylab import *
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.models import LinearColorMapper
from bokeh.models import HoverTool

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

from textblob import TextBlob
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

from time import time

  

     |████████████████████████████████| 18.5 MB 349 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
panel 0.12.1 requires bokeh<2.4.0,>=2.3.0, but you have bokeh 2.4.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=e116a94a169930667852ae18985c3c07f4d76e99bf34441c42bf5f480f01db47
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/

/content/drive/My Drive/Colab Notebooks


In [ ]:
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def predict_aspect(text):
  # Importing the required libraries
  import spacy
  sp = spacy.load("en_core_web_sm")
  from textblob import TextBlob

  # Creating a list of positive and negative sentences.
  mixed_sen = [text]

  # An empty list for obtaining the extracted aspects
  # from sentences.
  ext_aspects = []

  # Performing Aspect Extraction
  for sen in mixed_sen:
    important = sp(sen)
    descriptive_item = ''
    target = ''
    for token in important:
      if token.dep_ == 'nsubj' and token.pos_ == 'NOUN':
        target = token.text
      if token.pos_ == 'ADJ':
        added_terms = ''
        for mini_token in token.children:
          if mini_token.pos_ != 'ADV':
            continue
          added_terms += mini_token.text + ' '
        descriptive_item = added_terms + token.text 
    ext_aspects.append({'aspect': target,'description': descriptive_item})

  print("ASPECT EXTRACTION\n")
  print(ext_aspects)


  for aspect in ext_aspects:
    aspect['sentiment'] = TextBlob(aspect['description']).sentiment

  print("\n")
  print("\nSENTIMENT ASSOCIATION\n")
  print(ext_aspects)


In [ ]:
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model

In [ ]:
# loading
with open('tokenizer_sentiment.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:

# Load model

def predict_sentiment(text):
    '''Function to predict sentiment class of the passed text'''
    model = load_model('timesofindia_lstm_model_for_sentiment.h5')

    sentiment_classes = ['Negative', 'Positive']
    max_len=50
    
    # Transforms text to a sequence of integers using a tokenizer object
    xt = tokenizer.texts_to_sequences(text)
    # Pad sequences to the same length
    xt = pad_sequences(xt, padding='post', maxlen=max_len)
    # Do the prediction using the loaded model
    yt = model.predict(xt).argmax(axis=1)
    # Print the predicted sentiment
    print('The predicted sentiment is', sentiment_classes[yt[0]])

    

    

   

In [ ]:
from keras.models import load_model

# Load model
model = load_model('timesofindia_lstm_model_for_topic.h5')

def predict_topic(text):
    '''Function to predict Topic class of the passed text'''
    model = load_model('timesofindia_lstm_model_for_topic.h5')

    topic_classes = ['Topic 1', 'Topic 2', 'Topic 3', 'Topic 4', 'Topic 5', 'Topic 6', 'Topic 7', 'Topic 8', 'Topic 9', 'Topic 10', 'Topic 11', 'Topic 12','Topic 13', 'Topic 14', 'Topic 15', 'Topic 16', 'Topic 17', 'Topic 18', 'Topic 19', 'Topic 20', 'Topic 21', 'Topic 22', 'Topic 23', 'Topic 24']
    max_len=50
    
    # Transforms text to a sequence of integers using a tokenizer object
    xt = tokenizer.texts_to_sequences(text)
    # Pad sequences to the same length
    xt = pad_sequences(xt, padding='post', maxlen=max_len)
    # Do the prediction using the loaded model
    yt = model.predict(xt).argmax(axis=1)
    # Print the predicted sentiment
    print('\nThe predicted Topic is', topic_classes[yt[0]])

    

    #for apspect based sentiment prediction
    # converting list to string type
    str1 = " " 
    
    # return string  
    txt= str1.join(text)
    predict_aspect(txt)

   

In [ ]:
def predict_sent_topic(text):
  get_txt=text

  predict_sentiment(get_txt)
  predict_topic(get_txt)




In [ ]:
predict_sent_topic(['The bomb on plane threat creates flutter at Bengaluru airport'])

The predicted sentiment is Negative

The predicted Topic is Topic 10
ASPECT EXTRACTION

[{'aspect': 'bomb', 'description': ''}]



SENTIMENT ASSOCIATION

[{'aspect': 'bomb', 'description': '', 'sentiment': Sentiment(polarity=0.0, subjectivity=0.0)}]


In [ ]:
predict_sent_topic(['India to see major growth in manufacturing sector in the year 2022'])

The predicted sentiment is Positive

The predicted Topic is Topic 3
ASPECT EXTRACTION

[{'aspect': '', 'description': 'major'}]



SENTIMENT ASSOCIATION

[{'aspect': '', 'description': 'major', 'sentiment': Sentiment(polarity=0.0625, subjectivity=0.5)}]


In [ ]:
predict_sent_topic(['ISROs sslv missile launch failed'])

The predicted sentiment is Negative

The predicted Topic is Topic 15
ASPECT EXTRACTION

[{'aspect': 'launch', 'description': ''}]



SENTIMENT ASSOCIATION

[{'aspect': 'launch', 'description': '', 'sentiment': Sentiment(polarity=0.0, subjectivity=0.0)}]
